In [1]:
#Peter Storm
#pqs4@cornell.edu
#Example numbers adapted from Josue Medellin-Azuara & Richard Howitt

In [2]:
import scipy.optimize
from ipywidgets import *
%matplotlib inline
from numpy import *
import matplotlib.pyplot as plt


In [3]:
def pmp_example_sliders(corn_price=250, corn_initial_acres=200, x_axis_scale=350):
    ###example from 
    ###Data Set###
    ###Region 1###
    #data are arranged [alfalfa, vine, corn]
    #price is $/ton
    crop_prices = [132, 700, corn_price]
    #yield in ton/acre
    crop_yields = [7, 6.5, 6]
    #costs are $/acre
    crop_costs = [681, 3478, 1000]
    ###Observed Data###
    #data are arranged [alfalfa, vine, corn]
    #acres of crops watered
    irrigated_crop_acres = [100, 30, corn_initial_acres]
    total_irrigated = sum(irrigated_crop_acres)
    total_available_land = total_irrigated
    #water application rates (ft per unit area)
    applied_water_rates = [4, 1.5, 2.5]


    #loops to populate additional information
    #revenues from crops, $/acre
    crop_revenues = []
    #total applied water, acre-ft
    applied_water = []
    #observed net returns for a given crop ($), acres * (revenue - cost)
    obs_net_returns = []
    net_return_per_acre = []
    for i in range(len(crop_prices)):
        crop_revenues.append(crop_prices[i] * crop_yields[i])
        applied_water.append(applied_water_rates[i] * irrigated_crop_acres[i])
        obs_net_returns.append(irrigated_crop_acres[i] * (crop_revenues[i] - crop_costs[i]))
        net_return_per_acre.append(crop_revenues[i] - crop_costs[i])

    total_applied_water = sum(applied_water)


    #create calibration constraints for solver
    #Decouples the calibration from constraints to ensure a unique outcome (Howitt, 1995)
    cal_mult = 1.0001
    cal_acre_constraints = []
    for i in range(len(irrigated_crop_acres)):
        cal_acre_constraints.append(irrigated_crop_acres[i] * cal_mult)

    #define constraints for model
    #note that for inequalities, the sum is to be calibrated to greater than or equal to 0
    #So we need to make sure that extra water or land remains, so contraint minus the calibrated amount
    def irr_acres_constr(cal_irr_acres, 
                         total_irrigated_water=total_irrigated):
        #limits the total irrigated acres in model to calibrated total
        return total_irrigated_water - sum(cal_irr_acres)
    def applied_water_constr(cal_irr_acres, 
                             applied_water_rates=applied_water_rates, 
                             total_applied_water=total_applied_water):
        #limits water applied in calibration to total applied in data
        total_water = 0
        for i in range(len(applied_water_rates)):
            total_water += applied_water_rates[i] * cal_irr_acres[i]
        return total_applied_water - total_water

    #need to create number of constraints based on number of crops
    def crop0(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[0] - cal_irr_acres[0]

    def crop1(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[1] - cal_irr_acres[1]

    def crop2(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[2] - cal_irr_acres[2]

    #create dictionary of constraints
    cons = [{'type':'ineq', 'fun': irr_acres_constr},
            {'type':'ineq', 'fun': applied_water_constr},
            {'type':'ineq', 'fun': crop0},
            {'type':'ineq', 'fun': crop1},
            {'type':'ineq', 'fun': crop2},
           ]

    #establish parameters for model
    guess_irr_acres = [50, 50, 50]

    #bounds for model
    bnds = ((0, total_available_land), (0, total_available_land), (0, total_available_land))


    def calc_observed_net_revenue(guess_irr_acres, applied_water_rates=applied_water_rates):
        """
        Calculates the total net revenues for all crops, negative since minimizing
        Inputs need to be lists of equal lengths with respective crops lined up
        """
        total_net_revenue = 0
        for i in range(len(applied_water_rates)):
            total_net_revenue += net_return_per_acre[i] * guess_irr_acres[i]

        return -1 * total_net_revenue

    #run the model to fit parameters
    results = scipy.optimize.minimize(calc_observed_net_revenue, #function to minimize
                                      x0=guess_irr_acres,
                                      method='SLSQP',
                                      bounds=bnds,
                                      constraints=cons)


    ###shadow price crop 0###

    def crop0(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[0] - cal_irr_acres[0] + 1

    def crop1(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[1] - cal_irr_acres[1]

    def crop2(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[2] - cal_irr_acres[2]

    #create dictionary of constraints
    cons = [{'type':'ineq', 'fun': irr_acres_constr},
            {'type':'ineq', 'fun': applied_water_constr},
            {'type':'ineq', 'fun': crop0},
            {'type':'ineq', 'fun': crop1},
            {'type':'ineq', 'fun': crop2},
           ]

    #establish parameters for model
    guess_irr_acres = [50, 50, 50]

    #bounds for model
    bnds = ((0, total_available_land), (0, total_available_land), (0, total_available_land))

    results0 = scipy.optimize.minimize(calc_observed_net_revenue, #function to minimize
                                      x0=guess_irr_acres,
                                      method='SLSQP',
                                      bounds=bnds,
                                      constraints=cons)
    shadow_0 = results.fun - results0.fun                    

    ###shadow price crop 1###

    def crop0(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[0] - cal_irr_acres[0] 

    def crop1(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[1] - cal_irr_acres[1] + 1

    def crop2(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[2] - cal_irr_acres[2]

    #create dictionary of constraints
    cons = [{'type':'ineq', 'fun': irr_acres_constr},
            {'type':'ineq', 'fun': applied_water_constr},
            {'type':'ineq', 'fun': crop0},
            {'type':'ineq', 'fun': crop1},
            {'type':'ineq', 'fun': crop2},
           ]

    #establish parameters for model
    guess_irr_acres = [50, 50, 50]

    #bounds for model
    bnds = ((0, total_available_land), (0, total_available_land), (0, total_available_land))

    results1 = scipy.optimize.minimize(calc_observed_net_revenue, #function to minimize
                                      x0=guess_irr_acres,
                                      method='SLSQP',
                                      bounds=bnds,
                                      constraints=cons)
    shadow_1 = results.fun - results1.fun       

    ###shadow price crop 2###

    def crop0(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[0] - cal_irr_acres[0] 

    def crop1(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[1] - cal_irr_acres[1] 

    def crop2(cal_irr_acres, cal_acre_constraints=cal_acre_constraints):
        return cal_acre_constraints[2] - cal_irr_acres[2] + 1

    #create dictionary of constraints
    cons = [{'type':'ineq', 'fun': irr_acres_constr},
            {'type':'ineq', 'fun': applied_water_constr},
            {'type':'ineq', 'fun': crop0},
            {'type':'ineq', 'fun': crop1},
            {'type':'ineq', 'fun': crop2},
           ]

    #establish parameters for model
    guess_irr_acres = [50, 50, 50]

    #bounds for model
    bnds = ((0, total_available_land), (0, total_available_land), (0, total_available_land))

    results2 = scipy.optimize.minimize(calc_observed_net_revenue, #function to minimize
                                      x0=guess_irr_acres,
                                      method='SLSQP',
                                      bounds=bnds,
                                      constraints=cons)
    shadow_2 = results.fun - results2.fun  

    lagrange_mults = [shadow_0, shadow_1, shadow_2]
    #calculate PMP parameters
    alpha = []
    gamma = []

    for i in range(len(lagrange_mults)):
        alpha.append(crop_costs[i] - lagrange_mults[i])
        gamma.append(2 * lagrange_mults[i] / irrigated_crop_acres[i])


    ##Solve calibrated model##

    def pmp_net_revenue(guess_irr_acres, alpha=alpha, gamma=gamma,
                        applied_water_rates=applied_water_rates, crop_revenues=crop_revenues):
        """
        Calculates the total net revenues for all crops using PMP, negative since minimizing
        Inputs need to be lists of equal lengths with respective crops lined up
        """
        revenue = 0
        pmp_cost = 0
        for i in range(len(guess_irr_acres)):
            revenue += crop_revenues[i] * guess_irr_acres[i] 
            pmp_cost += alpha[i] * guess_irr_acres[i] + 0.5 * gamma[i] * guess_irr_acres[i] ** 2
        return (pmp_cost - revenue)

    #setting constraints
    water_available = 945 #in acre-ft
    land_available = 330 #in acres
    def irr_acres_constr(cal_irr_acres, land_available=land_available):
        #limits the total irrigated acres to land available

        return land_available - sum(cal_irr_acres)

    def applied_water_constr(cal_irr_acres, applied_water_rates=applied_water_rates, water_available=water_available):
        #limits water applied in calibration to total applied in data
        total_water = 0
        for i in range(len(applied_water_rates)):
            total_water += applied_water_rates[i] * cal_irr_acres[i]

        return water_available - total_water

    #create dictionary of constraints
    cons = [{'type':'ineq', 'fun': irr_acres_constr},
            {'type':'ineq', 'fun': applied_water_constr}]

    #establish parameters for model
    guess_irr_acres = [50, 50, 50]

    #bounds for model
    bnds = ((0, land_available), (0, land_available), (0, land_available))

    #run calibrated model
    pmp_results = scipy.optimize.minimize(pmp_net_revenue, #function to minimize
                                      x0=guess_irr_acres,
                                      method='SLSQP',
                                      bounds=bnds,
                                      constraints=cons)

    alfalfa_params = [alpha[0], gamma[0]]
    vine_params = [alpha[1], gamma[1]]
    corn_params = [alpha[2], gamma[2]]


    #set the x-axis limits
    t = linspace(0, x_axis_scale)

    #MC is derivative of total cost function
    corn_MC = corn_params[0]  + corn_params[1] * t 
    #marginal benefits is simply the revenue per acre
    corn_MB = crop_revenues[2] + 0 * t


    plt.figure(dpi=135)
    plt.ylabel('Marginal Cost ($/acre)')
    plt.xlabel('Acres of Corn Planted')


    plt.plot(t, corn_MC, 'r', label='Marginal Cost')
    plt.plot(t, corn_MB, 'b', label='Marginal Revenue')


    plt.tick_params(labelsize=8)

    legend = plt.legend(loc=4, shadow=True)

    plt.show()
    print("Corn Alpha: ", round(corn_params[0], 2))
    print("Corn Gamma: ", round(corn_params[1], 2))

In [5]:
slider = interactive(pmp_example_sliders, corn_price=(10, 500, 10), 
                     corn_initial_acres=(0.01,400, 10), x_axis_scale=(30, 650, 10))
display(slider)

interactive(children=(IntSlider(value=250, description='corn_price', max=500, min=10, step=10), FloatSlider(va…